# Neuroevolution with NEAT

We will start by installing the ```neat-python``` library, which provides an implementation of NEAT, and the ```networkx``` library to draw the resulting network

In [ ]:
#%pip install neat-python

In [ ]:
import neat
import random
import networkx as nx

The problem is a variant of the XOR problem with one additional input.

We have three inputs $x_1, x_2, x_3$ and two outputs $y_1 = x_1 \oplus x_2$ and $y_2 = \neg x_3$.

Part of the following code is adapted from a ```neat-python``` example

In [ ]:
xor_inputs = [(0.0, 0.0, 0.0), (0.0, 0.0, 1.0),
              (0.0, 1.0, 0.0), (0.0, 1.0, 1.0),
              (1.0, 0.0, 0.0), (1.0, 0.0, 1.0),
              (1.0, 1.0, 0.0), (1.0, 1.0, 1.0)]
xor_outputs = [(0.0, 1.0), (0.0, 0.0),
               (1.0, 1.0), (1.0, 0.0),
               (1.0, 1.0), (1.0, 0.0),
               (0.0, 1.0), (0.0, 0.0)]

To define how to eval the genomes we generate the network starting from the genome using ```nn.FeedFOrwardNetwork.create```, and we can use the method ```activate``` to compute the outputs of the net.

In [ ]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 16.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for xi, xo in zip(xor_inputs, xor_outputs):
            output = net.activate(xi)
            genome.fitness -= (output[0] - xo[0]) ** 2
            genome.fitness -= (output[1] - xo[1]) ** 2

The ```neat-python``` library **requires** a (non-trivial) configuration file (see ```neat_config.txt```)

In [ ]:
config = neat.Config(neat.genome.DefaultGenome,
                     neat.reproduction.DefaultReproduction,
                     neat.species.DefaultSpeciesSet,
                     neat.stagnation.DefaultStagnation,
                     "neat_config.txt")

We can now create the initial population (based on the given configuration)

In [ ]:
population = neat.Population(config)

We evolve the population from $500$ generations

In [ ]:
random.seed(0)
winner = population.run(eval_genomes, 5000)

We can print the best network found:

In [ ]:
print(f'Best genome:\n{winner}')

And the outputs of the network:

In [ ]:
print('\nOutput:')
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
for xi, xo in zip(xor_inputs, xor_outputs):
    output = winner_net.activate(xi)
    print(f"input {xi}, expected output {xo}, got {output}")

FInally, we can also draw a graphical representation of the network

In [ ]:
n_outputs = 2
n_inputs = 3
g = nx.DiGraph()
for name in range(-n_inputs, 0):
    g.add_node(name, node_type=0)
for name in winner.nodes.keys():
    if name < n_outputs:
        node_type = 2
    else:
        node_type = 1
    g.add_node(name, node_type=node_type)
for i,j in winner.connections.keys():
    g.add_edge(i, j)
pos = nx.multipartite_layout(g, subset_key="node_type")
nx.draw(g, pos=pos)